# Import the robust portfolio optimization library 

In [2]:
include("PortfolioModels.jl");
include("PortfolioMetrics.jl");

# Load and display the returns series of the Dow-Jones stocks

In [135]:
# Read the Dow Jones stock returns data
rets_df = CSV.read(path*"dowj_stock_rets.csv", DataFrame);

# Read the tickers
tickers = names(select(rets_df, Not(:Date)));

# Read the returns matrix
rets = Matrix(select(rets_df, Not(:Date)));

# Display a sample of the data
pretty_table(last(rets_df, 12), title = "Daily Returns of the Dow-Jones Stocks")

Daily Returns of the Dow-Jones Stocks
┌────────────┬─────────────┬─────────────┬─────────────┬─────────────┬──────────────┬──────────────┬─────────────┬─────────────┬──────────────┬─────────────┬─────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬──────────────┬──────────────┬─────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
│       Date │        AAPL │        AMGN │        AMZN │         AXP │           BA │          CAT │         CRM │        CSCO │          CVX │         DIS │          GS │           HD │          HON │         IBM │         JNJ │         JPM │          KO │          MCD │          MMM │         MRK │         MSFT │          NKE │        NVDA │          PG │         SHW │          TRV │          UNH │           V │          VZ │         WMT │
│       Date │     Float64 │     Float64 │     Float64 │     Float64 │

# Illustrate some portfolio allocation models

In [ ]:
# Compute the optimization parameters
mean_rets = mean(rets, dims=1);
cov_matrix_rets = cov(rets);

### Equally weighted portfolio

In [87]:
# Compute the weights of EW portfolio
n = size(rets,2)
w = ones(n)./n;

#Compute the mean return of the EW portfolio
ew_ret = dot(w, mean_rets);

# Print the mean returns and EW portfolio weights
print("Annualized mean return of the EW portfolio: \n", round((1 + ew_ret)^252-1, digits=2))

Annualized mean return of the EW portfolio: 
0.19

### Mean-Variance Model

\begin{align*}
     \min_{\mathbf{w}} &\quad \mathbf{w}^\top \hat{\Sigma} \mathbf{w} \\
     \text{s.t.}&\quad \mathbf{\hat{\mu}^\top w} \geq \mu^*\\
     &\quad \mathbf{1}^\top \mathbf{w} = 1\\
     &\quad\mathbf{w} \geq \bf 0
\end{align*}


- $\mathbf{w}$: the portfolio allocation weights
- $\hat{\mu}$: estimated expected return 
- $\hat{\Sigma}$: estimated covariance matrix of the returns
- $\mu^* $: minimum target return level 


In [ ]:
# Set the target return
target_ret = 0.001

# Compute the minimum variance portfolio weights
w, mv_ret, _, _ = compute_mv_weights(mean_rets, cov_matrix_rets, target_ret);

# Print the weights of the MV portfolio
pretty_table(w[w.!=0], header = ["weights"],    
            row_names=tickers[(w.!=0) .==1], 
            title = "Recommended allocation of the MV portfolio:")

Recommended Allocation of the MV portfolio:
┌──────┬──────────┐
│      │  weights │
├──────┼──────────┤
│ AAPL │ 0.263378 │
│ NVDA │  0.14906 │
│  SHW │ 0.242564 │
│  WMT │ 0.344999 │
└──────┴──────────┘


In [83]:
# Print the weights of the MV portfolio
print("Annualiazed mean return of the MV portfolio:\n ", round((1 + mv_ret)^252-1, digits=2))

Annualiazed mean return of the MV portfolio:
 0.29

### Conditional Value-at-Risk Model

\begin{align*}
    \min_{\mathbf{w}}& \quad \text{CVaR}_{\beta}(\mathbf{w})\\    
    &\quad \mathbf{\hat{\mu}^\top w} \geq \mu^*\\
    &\quad\mathbf{1}^\top \mathbf{w} = 1\\
    &\quad \mathbf{w}\geq0
\end{align*}

- $\mathbf{w}$: the portfolio allocation weights
- $\hat{\mu}$: estimated expected return 
- $\mu^* $: minimum target return level 
- $\beta$: the confidence level


where $\displaystyle \text{CVaR}_\beta(\mathbf{w}):=\frac{1}{1-\beta}\int_{f(\mathbf{ w, r})\geq\alpha_\beta(\mathbf{w})}f(\mathbf{w, r})p(\mathbf{r})d\bf r$


- $\bf r$: random assets returns
- $p(\mathbf{r})$: the density function of the returns
- $f(\mathbf{ w, r})$: the portfolio loss function
- $\alpha_\beta(\mathbf{w})$: the value at risk $\text{VaR}_\beta$ for the portfolio $\bf w$


In [ ]:
# Set the beta for the CVaR portfolio
beta = 0.95;

# Compute the CVaR portfolio weights
target_ret = 0.0015;

# Compute the CVaR portfolio weights
w, cvar_ret, alpha, _, _ = compute_cvar_weights(rets, mean_rets, beta, target_ret);
 
# Print the weights of the CVaR portfolio
pretty_table(w[w.!=0], header = ["weights"], 
             row_names=tickers[(w.!=0) .==1],
             title = "Recommended allocations of the CVaR portfolio:")

Recommended Allocations of the CVaR portfolio:
┌──────┬───────────┐
│      │   weights │
├──────┼───────────┤
│ AAPL │  0.202388 │
│ AMZN │  0.149841 │
│ NVDA │  0.516593 │
│  SHW │  0.119799 │
│  UNH │ 0.0113786 │
└──────┴───────────┘


In [114]:
# Print the mean return of CVaR portfolio
print("Annualized mean return of the CVaR portfolio:\n ", round((1 + cvar_ret)^252-1, digits=2))


Annualized mean return of the CVaR portfolio:
 0.46

### Omega Ratio Model

$$
\begin{matrix}
\displaystyle
    \max &  \displaystyle\Omega_\tau(\mathbf{w})\\
    \text{s. t.}& \mathbf{1}^\top\mathbf{w} = 1\\
    &\mathbf{w} \geq 0 
   \end{matrix}
$$

- $\mathbf{ w}$: vector of portfolio weights.

- $\tau$: threshold level that distinguishes gains and losses.


where $\displaystyle\Omega_\tau(\mathbf{w}) = \frac{\int_\tau^{+\infty}[1-F(r)]dr}{\int_{-\infty}^\tau F(r)dr} 
$
- $\mathbf{r} $: vector of asset returns.
- $F( r)$ is the cumulative distribution function of the portfolio returns



In [132]:
# Set the parameters for the Omega-Ratio portfolio
tau = 0.0;
delta_range = collect(0.6:0.05:0.85);

# Compute the Omega-Ratio portfolio weights
delta, w, omega_ret, mean_shortcom, excess_shortcom_tradeoff, output = compute_omega_ratio_weights(rets, mean_rets, tau, delta_range);

# Print the weights of the Omega- Ratio portfolio
pretty_table(w[w.!=0], header = ["weights"], 
             row_names=tickers[(w.!=0) .==1],
              title = "Recommended allocations of the Omega-Ratio portfolio:")

Recommended allocations of the Omega-Ratio portfolio:
┌──────┬────────────┐
│      │    weights │
├──────┼────────────┤
│ AAPL │   0.235595 │
│ AMZN │   0.108931 │
│   HD │   0.059638 │
│ NVDA │   0.278157 │
│  SHW │   0.125745 │
│  UNH │   0.142149 │
│    V │    0.04925 │
│  WMT │ 0.00053504 │
└──────┴────────────┘


In [133]:
# Print the weights of the Omega-Ratio portfolio
print("Annualized mean return of the Omega-Ratio portfolio:\n ", round((1 + omega_ret)^252-1, digits=2))

Annualized mean return of the Omega-Ratio portfolio:
 0.38

# Illustrate some robust optimization models

### Mean-Variance with Box-Uncertainty model
$$
    \max_{\mathbf{w}}\min_{\mu\in\mathcal{U}}\quad\mu^\top\mathbf{w} - \delta\mathbf{w}^\top\hat{\Sigma}\mathbf{w}\\
    \text{s.t. }\quad \mathbf{1}^\top\mathbf{w}=1\\
    \quad\quad \mathbf{w}\geq 0
$$


- $\mathbf{w}$: the portfolio allocation weights
- $\mu$: the true mean returns
- $\hat{\Sigma}$: estimated covariance matrix of the returns
- $\delta$: risk-aversion parameter


- $
    \mathcal{U}:= \{ \mu \mid |\mu - \hat{\mu}| \leq \varepsilon \}
$ captures the uncertainty in the estimation of the mean

- $\hat{\mu}$: the estimated mean
- $\varepsilon$: upper bounds on the deviation of each asset’s expected return from its estimated value.  For a $95\%$ confidence interval around the mean, $\varepsilon=\frac{1.96s}{\sqrt{T}}$, $T$ is the sample size used in the estimation, $s$ represents the estimated standard deviations of the returns


In [ ]:
# Set model parameters for the MV-Box portfolio
alpha = 0.05;
target_ret = 0.0008;

# Compute the MV-Box portfolio weights
w, mvbu_ret, _, _ = compute_mvbu_weights(rets, mean_rets, cov_matrix_rets, target_ret, alpha);

# Print the weights of the MV-Box portfolio
pretty_table(w[w.!=0], header = ["weights"], row_names=tickers[(w.!=0) .==1], title = "Recommended allocations of the MV-Box model:")

Recommended Allocations of the MV-Box model:
┌──────┬───────────┐
│      │   weights │
├──────┼───────────┤
│ AAPL │  0.194033 │
│ AMZN │ 0.0924345 │
│   KO │  0.254527 │
│ NVDA │  0.079742 │
│  WMT │  0.379263 │
└──────┴───────────┘


In [105]:
# Print the weights of the MV-Box portfolio
print("Annualized mean return of the MV box-uncertainty model:\n ", round((1 + mvbu_ret)^252-1, digits=2))

Annualized mean return of the MV box-uncertainty model:
 0.22

## Mean-Variance Ellipsoid Uncertainty

$$
    \max_{\mathbf{w}}\min_{\mu\in\mathcal{U}}\quad\mu^\top\mathbf{w} - \delta\mathbf{w}^\top\Sigma\mathbf{w}\\
    \text{s.t. }\quad \mathbf{1}^\top\mathbf{w}=1\\
    \quad\quad \mathbf{w}\geq 0\\
$$


- $\mathbf{w}$: the portfolio allocation weights
- $\mu$: the true mean returns
- $\hat{\Sigma}$: estimated covariance matrix of the returns
- $\delta$: risk-aversion parameter


- $
    \mathcal{U} = \{ \mu \mid (\mu - \hat{\mu})^\top \Sigma_\mu^{-1} (\mu - \hat{\mu}) \leq \varepsilon^2 \}
$ captures the uncertainty in the estimation of the mean returns

- $\hat{\mu}$: the estimated mean

- $\Sigma_\mu$: covariance matrix of the estimation errors in the mean returns
- $\varepsilon$:  $\alpha$-th percentile of the $\chi^2$ distribution with $N$ degrees of freedom, $\alpha$ is the confidence level, $N$ the number of assets

In [137]:
# Set the model parameters for the MVEU portfolio
alpha = 0.95;
target_ret = 0.001;

# Compute the optimization paramters for the MVEU portfolio
std_devs = std(rets, dims=1);
cov_matrix_mean_estimation = Diagonal(std_devs[:]);

# Compute the MVEU portfolio weights    
w, mveu_ret, _ , _ = compute_mveu_weights(mean_rets, cov_matrix_rets, cov_matrix_mean_estimation, target_ret, alpha);

# Print the weights of the MV-Box portfolio
print('\n')
pretty_table(w[w.>1e-3], header = ["weights"], row_names=tickers[(w.>1e-3) .==1], title = "Recommended allocations  of the MV elliptic-uncertainty model:")


Set parameter Username
Set parameter LicenseID
Academic license - for non-commercial use only - expires 2025-11-19

Recommended allocations  of the MV elliptic-uncertainty model:
┌──────┬────────────┐
│      │    weights │
├──────┼────────────┤
│ AAPL │  0.0968593 │
│ AMGN │  0.0337127 │
│ AMZN │  0.0886163 │
│  AXP │  0.0386972 │
│   BA │  0.0122292 │
│  CAT │   0.031056 │
│  CRM │  0.0631099 │
│ CSCO │ 0.00293575 │
│  CVX │  0.0018772 │
│  DIS │ 0.00903736 │
│   GS │  0.0190596 │
│   HD │     0.0658 │
│  HON │   0.019965 │
│  JPM │  0.0376832 │
│  MCD │  0.0300055 │
│  MRK │ 0.00396438 │
│ MSFT │  0.0617149 │
│  NKE │  0.0221313 │
│ NVDA │   0.116529 │
│  SHW │  0.0682067 │
│  TRV │  0.0285598 │
│  UNH │  0.0557642 │
│    V │   0.068112 │
│  WMT │  0.0243741 │
└──────┴────────────┘


In [138]:
# Print the weights of the MVEU portfolio
print("Annualized mean return of the MV-Box model:\n ",  round((1 + mveu_ret)^252-1, digits=2));

Annualized mean return of the MV-Box model:
 0.29

## Backtesting and Performance

The models have been tested over the period spanning 2009-2024. The performance metrics have been averaged over the rolling out-of-sample test windows.

In [ ]:
rets = rets_df;

# Generate the testing data windows
testing_data = compute_rolling_test_data(rets, Date("2009-01-01"), Date("2024-12-30"));

# Collect the performance metrics of running the models on the testing data
perf_results_long = innerjoin(evaluate_models_perf(rets, testing_data), 
                              compute_folios_turnover(testing_data), 
                              on = [:testing_start, :testing_end, :model]);

# Generate the reporting on the results
perf_summary = compute_perf_summary(perf_results_long);

### Results for the porfolios composition metrics 



In [ ]:
# Print the portfolio composition metrics in a pretty table format
pretty_table(last(select(perf_summary, Not(:Metric)),3), 
                    row_names= ["Mean Holding #", "HHI index", "Turnover"], 
                    title = "Composition of the different portfolio models");

Composition of the different portfolio models
┌────────────────┬──────────┬──────────┬──────────┬───────────┬──────────┐
│                │    Omega │     CVaR │       MV │      MVEU │     MVBU │
│                │  Float64 │  Float64 │  Float64 │   Float64 │  Float64 │
├────────────────┼──────────┼──────────┼──────────┼───────────┼──────────┤
│ Mean Holding # │  5.37288 │  8.83051 │  1.30508 │      30.0 │  1.32203 │
│      HHI index │ 0.354548 │ 0.219771 │ 0.939582 │  0.035874 │ 0.922519 │
│       Turnover │     1.27 │  1.38387 │ 0.544523 │ 0.0417387 │ 0.651105 │
└────────────────┴──────────┴──────────┴──────────┴───────────┴──────────┘


### Results for the porfolios performance metrics 


In [127]:
# Display the results for the composition of the portfolios
pretty_table(select(first(perf_summary,6), Not(:Metric)), 
                row_names=["Daily Return", "Daily Volatility", "Sharpe Ratio", "CVaR", "Omega-Ratio", "Sortino Ratio"],
                 title = "Portfolios Performance Summary: the values are the averages over the rolling out-of-sample tests");

Portfolios Performance Summary: the values are the averages over the rolling out-of-sample tests
┌──────────────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│                  │        Omega │         CVaR │           MV │         MVEU │         MVBU │
│                  │      Float64 │      Float64 │      Float64 │      Float64 │      Float64 │
├──────────────────┼──────────────┼──────────────┼──────────────┼──────────────┼──────────────┤
│     Daily Return │  0.000779847 │  0.000442004 │    0.0003581 │  0.000640242 │  0.000383941 │
│ Daily Volatility │    0.0129703 │     0.007702 │   0.00974963 │   0.00867521 │   0.00963383 │
│     Sharpe Ratio │    0.0857052 │     0.080967 │    0.0482553 │     0.107808 │    0.0495658 │
│             CVaR │ -0.000632922 │ -0.000364857 │ -0.000662713 │ -0.000259815 │ -0.000626994 │
│      Omega-Ratio │      1.32283 │      1.33128 │      1.21028 │      1.42119 │      1.20709 │
│    Sortino Ratio │     0.139726 │    